# Raku RAG demo

<span style="font-size: 16pt;">... and </span>
<span style="font-size: 16pt; font-style: italic;">Semantic Nearest Neighbors Graph</span>

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
[RakuForPrediction-book at GitHub](https://github.com/antononcube/RakuForPrediction-book)      
September 2024 


-----

## Introduction

The so called ***Retrieval Augmented Generation (RAG)*** functionalities of Large Language Models (LLMs) are demonstrated using semantic nearest neighbors graphs.

Vector Databases (VDBs) made with the Raku package ["LLM::RetrievalAugmentedGeneration"](https://raku.land/zef:antononcube/LLM::RetrievalAugmentedGeneration) are used.

1. Semantic graph for words 
    - Select a set of words
    - Find LLM embeddings for the those words
    - Find nearest neighbor graph for the obtained vectors

2. Nearest neighbor graph -- joined VDBs
    - Ingest VDBs corresponding to (long) podcast interviews
        - "Modern Wisdom" with [Eric Weinstein](https://en.wikipedia.org/wiki/Eric_Weinstein)
        - Eclectic and long interviews (≈3.5 hours each)
        - Heterogeneous content (wide variety of topics)
        - *(Hence, suitable for semantic similarities demos)*
    - Merge VDBs
    - Make a corresponding semantic graph
        - Nearest neighbor graph
        - Connected components

3. Nearest neighbor graph and RAG 
    - Create a semantic relationship graph for an interview
    - LLM-derivation of summaries for top connected components
    - Graph plot traces 
        - Using suitable graph tooltips
    - Do a retrieval based LLM summary for a query

-----

## Setup

In [ ]:
use LLM::Configurations;
use LLM::RetrievalAugmentedGeneration;
use LLM::RetrievalAugmentedGeneration::VectorDatabase;

use XDG::BaseDirectory :terms;

use Data::Importers;
use Data::Reshapers;
use Data::Summarizers;
use Math::Nearest;
use Math::DistanceFunctions::Native;
use Statistics::OutlierIdentifiers;

use NativeCall;

use Math::Nearest;
use Graph;
use JavaScript::D3;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [ ]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [ ]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [ ]:
my $title-color = 'Ivory';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

------

## Ingest vector databases

Ingest vector database from the default directory and show a summary:

In [ ]:
#% html
my @field-names = <id name item-count dimension version llm-service llm-embedding-model created>;

vector-database-objects(f=>'hash', :flat)

==> { $_.map({ $_<created> = $_<file>.IO.created.DateTime.Str.subst('T',' ').substr(^19); $_}).sort(*<created>).reverse }()
==> to-html(:@field-names)

Get specific databases:

In [ ]:
#my @vdbs = vector-database-objects(f=>'hash', :flat).grep({ $_<name> ∈ <No833 No747> && $_<llm-service> eq 'gemini' }).map({ create-vector-database(file => $_<file>) })

my @vdbs = vector-database-objects(f=>'hash', :flat).grep({ $_<id> ∈ <a840beb1-b0fa-49af-b206-25f4644daae0> }).map({ create-vector-database(file => $_<file>) })

Sample vectors:

In [ ]:
.say for @vdbs.head.vectors.pick(3)

Sample items:

In [ ]:
#% html
@vdbs.head.items.pick(3).map({ <key value> Z=> $_.kv })».Hash.Array 
==> to-html(field-names => <key value>, align => 'left')

-----

## Combined databases graph

Here we join the selected databases:

In [ ]:
#my $vdbObj2 = vector-database-join(@vdbs)
my $vdbObj2 = @vdbs.head


Find a nearest neighbors graph:

In [ ]:
my @edges2 = nearest-neighbor-graph($vdbObj2.vectors.pairs, 1, method => 'Scan', distance-function => &euclidean-distance, format => 'raku');

my $gr2 = Graph.new(@edges2)

In [ ]:
my @comps2 = $gr2.connected-components.sort(-*.elems).head(8);

@comps2.elems

In [ ]:
#% js
@edges2
==> js-d3-graph-plot(
        :$background,
        highlight => @comps2.map({[|$_, |$gr2.subgraph($_).edges]}),
        vertex-label-color => 'none',
        edge-thickness => 2,
        vertex-size => 4,
        vertex-color => 'Blue',
        width => 1100,
        height => 950,
        edge-color => 'Gray',
        vertex-color => 'DarkGray',
        #force => {charge => {strength => -3, iterations => 4}, collision => {radius => 14, iterations => 4}, link => {distance => 1};}
    )

-----

## Nearest neighbors for a query

Query:

In [ ]:
my $query = 'What are the problems with the newly introduced tariffs? Give a list of bullet points.'

In [ ]:
# Same configuration is used for the vector database creation
my $conf = llm-configuration("Gemini");

$conf.Hash.elems

Make a query vector:

In [ ]:
my $vec = llm-embedding($query, e => $conf).head».Num.Array;

deduce-type($vec)

Find the nearest neighbors:

In [ ]:
my @nns = |$vdbObj2.nearest($vec, 6).flat(:hammer)

Get the corresponding "paragraphs":

In [ ]:
my @paragraphs = $vdbObj2.items{|@nns};

.say for @paragraphs».&text-stats

Generate LLM based answer using the nearest neighbors:

In [ ]:
#% markdown

llm-synthesize([ 
    "Summarize the following text into a list of three-four points, each with no more than 8 words:",
    $vdbObj.items{|@nns.sort}.join(" "),
], e => $conf4o)

In [ ]:
#% markdown

llm-synthesize([ 
    "Answer the inquiry:",
    $query,
    "using the following text:",
    $vdbObj.items{|@nns.sort}.join(" "),
], e => $conf4o)

In [ ]:
llm-prompt-data(/:i SophisticatedFeedback /)

-----

## Extract wisdom

In [ ]:
my $res = llm-synthesize([ 
    llm-prompt('ExtractArticleWisdom')(),
    $vdbObj.items{|@nns.sort}.join(" "),
], e => $conf4o);

$res.&text-stats

In [ ]:
#% markdown
$res.subst( / ^^ '## '/, '### '):g

-------

## References

### Articles

[AA1] Anton Antonov, 
["Outlier detection in a list of numbers"](https://rakuforprediction.wordpress.com/2022/05/29/outlier-detection-in-a-list-of-numbers/),
(2022),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Packages

[AAp1] Anton Antonov,
[WWW::OpenAI Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::PaLM Raku package](https://github.com/antononcube/Raku-WWW-PaLM),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov,
[LLM::Prompts Raku package](https://github.com/antononcube/Raku-LLM-Prompts),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp5] Anton Antonov,
[ML::FindTextualAnswer Raku package](https://github.com/antononcube/Raku-ML-FindTextualAnswer),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp6] Anton Antonov,
[Math::Nearest Raku package](https://github.com/antononcube/Raku-Math-Nearest),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp7] Anton Antonov,
[Math::DistanceFunctions Raku package](https://github.com/antononcube/Raku-Math-DistanceFunctions),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp8] Anton Antonov,
[Statistics::OutlierIdentifiers Raku package](https://github.com/antononcube/Raku-Statistics-OutlierIdentifiers),
(2022),
[GitHub/antononcube](https://github.com/antononcube).

## Videos

[AAv1] Anton Antonov,
["Raku RAG demo"](https://www.youtube.com/watch?v=JHO2Wk1b-Og),
(2024),
[YouTube/AAA4prediction](https://www.youtube.com/@AAA4prediction).

[CWv1] Chris Williamson,
["Eric Weinstein - Why Does The Modern World Make No Sense? (4K)"](https://www.youtube.com/watch?v=p_swB_KS8Hw),
(2024),
[YouTube/@ChrisWillx](https://www.youtube.com/@ChrisWillx).   
([transcript](https://podscripts.co/podcasts/modern-wisdom/747-eric-weinstein-why-does-the-modern-world-make-no-sense).)

[CWv2] Chris Williamson,
["Eric Weinstein - Are We On The Brink Of A Revolution? (4K)"](https://www.youtube.com/watch?v=PYRYXhU4kxM),
(2024),
[YouTube/@ChrisWillx](https://www.youtube.com/@ChrisWillx).   
([transcript](https://podscripts.co/podcasts/modern-wisdom/833-eric-weinstein-are-we-on-the-brink-of-a-revolution).)